In [1]:
import pandas as pd
import sys
sys.path.append('./utils')
from utils import *
from rank import *

In [2]:
# Name master に含まれているデータを選択
table_name = 'V_RESTAURANT_NAME_MASTER'
columns= "all"
# データ保存先
data_folda = '../data/'

In [3]:
# snowflake から master data を読み込み
pd_sf = read_table(data_folda,table_name,columns,read_snowflake=False,col="*")

In [4]:
pd_sf.columns

Index(['restaurant_id', 'list_type_id', 'name', 'branch_name', 'address',
       'latitude', 'longitude', 'telephone', 'reservation_telephone', 'url',
       'original_id', 'available', 'sort_no', 'created', 'modified',
       'prefecture', 'area_name', 'business_status', 'closed_date'],
      dtype='object')

In [5]:
# original_id, available, closed_date, closed_flg のみを抽出
pd_sf = pd_sf[['restaurant_id','original_id', 'available', 'closed_date', 'business_status',"prefecture"]]

In [6]:
# original_id is not null and available is 1 
pd_sf=pd_sf[pd_sf['original_id'].isna()==False]
pd_sf["available"]=pd_sf["available"].astype(int)
pd_sf=pd_sf[pd_sf["available"]==1]
pd_sf['original_id']=pd_sf['original_id'].astype(int)

In [7]:
# close_date を日付に変換
pd_sf['closed_date'] = pd.to_datetime(pd_sf['closed_date'], format='%Y-%m-%d', errors='coerce')

In [8]:
# yyyymmddの列を追加する。
# 他のデータの内容は変わらず、2020年1月から2023年11月まで各月分のデータを用意する。
# 日付けは各月末日とする
# pandas のconcatは時間がかかるので、一旦、別に保存して最後に一度にconcatする。

eval_period=pd.date_range('2023-01-01', '2023-12-01', freq='MS')
#eval_period から一日前の日付を取得
eval_period=eval_period - pd.Timedelta(days=1)

# len(eval_period)のサイズを持つ、pandas のリストを作成
temp_=[pd.DataFrame() for x in range(len(eval_period))]


for i in eval_period:
    print(i)
    # pd_sfのコピーを作成
    pd_sf_copy=pd_sf.copy()
    # yyyymmddの列を追加、データの受領日を本来入れるべきであるが、ここでは1日を入れる
    pd_sf_copy['base_date']=i
    # augumented_dataに追加
    temp_[eval_period.get_loc(i)]=pd_sf_copy
    
# pseudo_time_series を作成
pseudo_time_series=pd.concat([temp_[i] for i in range(len(temp_))], ignore_index=True)


2022-12-31 00:00:00
2023-01-31 00:00:00
2023-02-28 00:00:00
2023-03-31 00:00:00
2023-04-30 00:00:00
2023-05-31 00:00:00
2023-06-30 00:00:00
2023-07-31 00:00:00
2023-08-31 00:00:00
2023-09-30 00:00:00
2023-10-31 00:00:00
2023-11-30 00:00:00


In [9]:
pseudo_time_series["base_date"]=pd.to_datetime(pseudo_time_series["base_date"], format='%Y-%m-%d', errors='coerce')

In [10]:
# target flg を追加する
# closed_date 該当月はflg=1とする
pseudo_time_series["closed_flg"]=0
pseudo_time_series.loc[pseudo_time_series["closed_date"].dt.strftime('%Y%m')==pseudo_time_series["base_date"].dt.strftime('%Y%m'),"closed_flg"]=1

In [11]:
# 同じrestaurant_idで、closed_flg=1のデータがある場合、その月以降はclosed_flg=1とする
pseudo_time_series.sort_values(["restaurant_id","base_date"],inplace=True)
pseudo_time_series["closed_flg"]=pseudo_time_series.groupby("restaurant_id")["closed_flg"].cummax()

In [12]:
# closed_flgをrestaurant_id毎に-3ヶ月ずらした列を追加する。これがtarget3になる。
pseudo_time_series["target3"]=pseudo_time_series.groupby("restaurant_id")["closed_flg"].shift(-3).ffill().astype(int)
# closed_flgをrestaurant_id毎に-6ヶ月ずらした列を追加する。これがtarget6になる。
pseudo_time_series["target6"]=pseudo_time_series.groupby("restaurant_id")["closed_flg"].shift(-6).ffill().astype(int)


In [13]:
# input は、closed_flgが1になる前の月までとする。
pseudo_time_series["eval_period"]=True
# augumented_data.loc[augumented_data.groupby("restaurant_id")["closed_flg"].shift(-1).ffill()==1,"eval_target3"]=False
# 課題：1日にモデルを作るが、モデル作成日に閉店した店舗は予測する必要があるか？
pseudo_time_series.loc[pseudo_time_series["base_date"]>=pseudo_time_series["closed_date"],"eval_period"]=False

In [14]:
pseudo_time_series["closed_flg"]=pseudo_time_series["closed_flg"].astype(int)

In [15]:
# 作成した時系列データを保存する
pseudo_time_series.to_pickle(data_folda+'pseudo_time_series.pkl')

In [16]:
master_data_20240301=pd.read_csv(data_folda+'restaurant_data.csv')

/tmp/ipykernel_4500/3375611667.py:1: DtypeWarning: Columns (59) have mixed types. Specify dtype option on import or set low_memory=False.
  master_data_20240301=pd.read_csv(data_folda+'restaurant_data.csv')


In [17]:
train_20231130=pseudo_time_series_with[pseudo_time_series_with["base_date"]==pd.datetime(2023,11,30)]
master_data_20240301["close_date"]=pd.to_datetime(master_data_20240301["close_date"],errors="coerce")
# business_status=0 もしくは、closed_dateが2023年11月30日以降、かつ掲載開始が11月30日以前の店舗の restaurant_id を取得
nonbaias_restaurant_id=master_data_20240301[((master_data_20240301["business_status"]==0)
                     |(master_data_20240301["close_date"]>"2023-11-30"))
                    &(master_data_20240301["created_at"]<="2023-11-30")]["id"]

# train_20231130のoriginal_idがnonbaias_restaurant_idに含まれるものを取得
train_20231130=train_20231130[train_20231130["original_id"].isin(nonbaias_restaurant_id)]
# その条件に合致するものは、eval_periodをTrueにする
train_20231130.loc[train_20231130["original_id"].isin(nonbaias_restaurant_id),"eval_period"]=True

# master_data_20240301 の中で 2023年12月1日から2024年2月29日に閉店した店舗の restaurant_id を取得
closed_in_3month_from20231201=master_data_20240301[(master_data_20240301["close_date"]>"2023-11-30")
                     &(master_data_20240301["close_date"]<"2024-03-01")
                     &(master_data_20240301["business_status"]==1)]["id"]

# train_20231130のoriginal_idがclosed_in_3month_from20231201に含まれるものについては、target3を1にする
train_20231130.loc[train_20231130["original_id"].isin(closed_in_3month_from20231201),"target3"]=1

train_20231130.to_pickle(data_folda+'pseudo_time_series_non_bias.pkl')

NameError: name 'pseudo_time_series_with' is not defined